<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Notebook_RiesgoCrediticio_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de Riesgo Crediticio con PySpark usando el dataset de Kaggle

Este cuaderno utiliza el conjunto de datos de crédito publicado en Kaggle por el UCI Machine Learning Repository.

- Dataset: [Default of Credit Card Clients Dataset](https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset)
- Archivo: `UCI_Credit_Card.csv`

## Instrucciones
1. Descargue manualmente el archivo desde Kaggle.
2. Cárguelo al entorno de ejecución (por ejemplo, Google Colab).
3. Ejecute las celdas paso a paso para entrenar y evaluar un modelo de clasificación con PySpark.

In [1]:
# Si está usando Google Colab, instale PySpark
!pip install pyspark

In [2]:
# Librerías de PySpark
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
# Crear SparkSession
spark = SparkSession.builder.appName("RiesgoCrediticioKaggle").getOrCreate()

### Subir archivo manualmente (`UCI_Credit_Card.csv`)
En Google Colab, puede usar el siguiente bloque para cargar el archivo:

In [4]:
from google.colab import files
uploaded = files.upload()

Saving UCI_Credit_Card.csv to UCI_Credit_Card (1).csv


### Cargar y explorar los datos
El dataset contiene 30,000 registros con variables socioeconómicas y de comportamiento financiero.

In [5]:
df = spark.read.csv("UCI_Credit_Card.csv", header=True, inferSchema=True)
df = df.withColumnRenamed("default.payment.next.month", "label")
df.select("LIMIT_BAL", "AGE", "EDUCATION", "MARRIAGE", "SEX", "label").show(5)

+---------+---+---------+--------+---+-----+
|LIMIT_BAL|AGE|EDUCATION|MARRIAGE|SEX|label|
+---------+---+---------+--------+---+-----+
|  20000.0| 24|        2|       1|  2|    1|
| 120000.0| 26|        2|       2|  2|    1|
|  90000.0| 34|        2|       2|  2|    0|
|  50000.0| 37|        2|       1|  2|    0|
|  50000.0| 57|        2|       1|  1|    0|
+---------+---+---------+--------+---+-----+
only showing top 5 rows



### Preparar pipeline de clasificación

In [6]:
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_idx")
    for col in ["EDUCATION", "MARRIAGE", "SEX"]
]
assembler = VectorAssembler(
    inputCols=["LIMIT_BAL", "AGE"] + [col+"_idx" for col in ["EDUCATION", "MARRIAGE", "SEX"]],
    outputCol="features_raw")
scaler = StandardScaler(inputCol="features_raw", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="label")
pipeline = Pipeline(stages=indexers + [assembler, scaler, lr])

In [7]:
# Entrenamiento del modelo
modelo = pipeline.fit(df)
predicciones = modelo.transform(df)
predicciones.select("label", "prediction", "probability").show(5)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    1|       0.0|[0.69400241111226...|
|    1|       0.0|[0.78484208412147...|
|    0|       0.0|[0.75881277160846...|
|    0|       0.0|[0.69929370572321...|
|    0|       0.0|[0.63602236063087...|
+-----+----------+--------------------+
only showing top 5 rows



### Evaluación del modelo

In [8]:
evaluador = BinaryClassificationEvaluator()
auc = evaluador.evaluate(predicciones)
print(f"Área bajo la curva (AUC): {auc:.4f}")

Área bajo la curva (AUC): 0.6223


### Conclusión
Este pipeline permite automatizar el análisis de riesgo crediticio utilizando datos reales. El uso de PySpark facilita su ejecución distribuida, y puede integrarse en entornos de producción con herramientas como Airflow y Kafka.